# Homework 2
#### Pierre-Antoine Desplaces, Anaïs Ladoy, Lou Richard

In [ ]:
# Import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import json
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Question 1

Obtain the 200 top-ranking universities in www.topuniversities.com ([ranking 2018](https://www.topuniversities.com/university-rankings/world-university-rankings/2018)). In particular, extract the following fields for each university: name, rank, country and region, number of faculty members (international and total) and number of students (international and total). Some information is not available in the main list and you have to find them in the [details page](https://www.topuniversities.com/universities/ecole-polytechnique-fédérale-de-lausanne-epfl).
Store the resulting dataset in a pandas DataFrame and answer the following questions:
- Which are the best universities in term of: (a) ratio between faculty members and students, (b) ratio of international students?
- Answer the previous question aggregating the data by (c) country and (d) region.

Plot your data using bar charts and describe briefly what you observed.


In [ ]:
URL1 = 'https://www.topuniversities.com/university-rankings/world-university-rankings/2018'
URL2 = 'http://timeshighereducation.com/world-university-rankings/2018/world-ranking'

In [ ]:
# Do the request
r = requests.get(URL1)

Inspecting the DOM of the website, we find that the ranking datas are stored in a text file at https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt as we can see here :

In [ ]:
# 357051.txt
id1 = r.text.find("357051.txt")
r.text[id1-88:id1+11]

We thus request the data from this URL, keeping only the first 200 elements.

In [ ]:
data_QS_URL = 'https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt'
data_QS = requests.get(data_QS_URL)
#create the dataframe
df_QS = pd.DataFrame(data_QS.json()['data'][:200])

We keep the relevant columns (Name of university, Rank, Score, Country, Region and university URL).

In [ ]:
df_ranking_qs = pd.DataFrame({"University" : df_QS.title,\
                           "Rank" : df_QS.rank_display,\
                           "Score" : df_QS.score.astype(float),\
                           "Country": df_QS.country,\
                           "Region": df_QS.region, "URL" : df_QS.url},\
                          columns = ['Rank', 'University', 'Score', 'Country', 'Region', 'URL'])

In [ ]:
df_ranking_qs.tail(10)

We observe that the 200th university has a rank = 201, this is because there is no 198th (we can see that the last 195th should be the 197th, and the next rank is 199).

Furthermore, as shown below, we have two universities present in the top-200 ranking that have missing values. We have checked manually their details page and indeed, these informations are missing. We decide to do nothing about these NaN values as it won't impact in a significant way our results.

In [ ]:
df_ranking_qs[df_ranking_qs.isnull().any(axis=1)]

We now have to get the number of faculties and students for each university. For this, we have to go the page of the university on the topuniversities website, and collect these information. Inspecting these webapges, we find that the number of faculty members is in the div number of the div total faculty, the number of international faculty members is in the div number of the div inter faculty, the number of students is in the div number of the div total students, and finally the number of international students is in the div number of the div total inter.

In [ ]:
# fetch the university informations from its webpage
def get_numbers_of(url):
    # get the data from the university page
    r = requests.get('https://www.topuniversities.com'+url)
    soup = BeautifulSoup(r.text, "html.parser")
    # find the corresponding tag
    try:
        staff = soup.find('div', class_="total faculty").find('div', class_="number").string
        staff = int(str(staff).replace('\n', "").replace(",", ""))
    except:
        staff = np.nan
    try :
        inter_staff = soup.find('div', class_="inter faculty").find('div', class_="number").string
        inter_staff = int(str(inter_staff).replace('\n', "").replace(",", ""))
    except:
        inter_staff = np.nan
    try : 
        students = soup.find('div', class_="total student").find('div', class_="number").string
        students = int(str(students).replace('\n', "").replace(",", ""))
    except:
        students = np.nan
    try : 
        inter_students = soup.find('div', class_="total inter").find('div', class_="number").string
        inter_students = int(str(inter_students).replace('\n', "").replace(",", ""))
    except:
        inter_students = np.nan
    
    return staff, inter_staff, students, inter_students

We fetch the results into new columns in our dataframe.

In [ ]:
df_ranking_qs['Total Faculty Members'], \
df_ranking_qs['International Faculty Members'], \
df_ranking_qs['Total Students'], \
df_ranking_qs['International Students'] = zip(*df_ranking_qs['URL'].map(get_numbers_of))

We can now remove the URL columns, as it is not useful anymore.

In [ ]:
df_ranking_qs = df_ranking_qs.drop('URL', axis=1)
df_ranking_qs.head()

#### a) Which are the best universities in term of ratio between faculty members and students ?

The ratio between the faculty members and students can be infer from the *Total Faculty Members* and *Total Students* columns.

To present our results, we decided to print the entire dataframe but sorted descendingly according the ratio *Faculty Members per Student* and then to plot the top-20 of universities for this parameter.

In [ ]:
df_ranking_qs['Fac Members per Student'] = df_ranking_qs['Total Faculty Members']/df_ranking_qs['Total Students']

In [ ]:
df_ranking_qs.sort_values('Fac Members per Student', ascending=False)

In [ ]:
def bars(t, data, xlab, ylab):
    sns.set_style('darkgrid')
    fig, ax = plt.subplots(figsize = (15,8))
    ax.set_title(t, fontsize=15, fontweight='bold')
    sns.barplot(x=xlab, y=ylab, data=data, saturation=0.7, errcolor='.7')
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
bars("Top-20 of the best universities in term of Faculty Members per Student for the QS ranking", df_ranking_qs.sort_values('Fac Members per Student', ascending=False)[:20], 'University', 'Fac Members per Student')

We observe that this ranking is different from the original ranking, but the 10 first university are almost all in the top 20 of the initial ranking.

#### b) Which are the best universities in term of ratio of international students?

In [ ]:
df_ranking_qs['% International Students'] = df_ranking_qs['International Students']/df_ranking_qs['Total Students']*100

In [ ]:
df_ranking_qs.sort_values('% International Students', ascending=False)

In [ ]:
bars("Top-20 of the best universities in term of International Students per University for the QS ranking", df_ranking_qs.sort_values('% International Students', ascending=False)[:20], 'University', '% International Students')

Here, the results are completely different from the original ranking. In the top 20 of the original ranking, 11 universities are from North America. Here, only one american university is in the top 20, and 15 are european universities, which makes sense as it is known that many european students go study in nearby countries in Europe.

#### c) Aggregating the data by country

Concerning the aggregation by country, we choose two types of representation.  
First we found, for each country present in the top-200 of the QS ranking, the best university in terms of *Faculty members per student* and *Percentage of International Students* and we stored the result in a dataframe sorted in a decreasing order according to the studied parameter.   
Secondly, we decided to plot the mean of these two variables for each country present in the top-200 as it seems an interesting way to visualize the results. We should precise that it's the mean of *Faculty members per student* and *Percentage of International Students* for the universities present in the top-200 ranking and not all the universities of the country.

##### - In terms of Ratio between Faculty members and Students

In [ ]:
qs_ratioFacStu_country=df_ranking_qs.loc[df_ranking_qs.groupby("Country")["Fac Members per Student"].idxmax()]
qs_ratioFacStu_country[['Country','University','Fac Members per Student']].set_index('Country').sort_values('Fac Members per Student',ascending=False).head()

The best university (QS top-200 ranking) in term of ratio between faculty members and students is the Caltech university in United States where there is one faculty members per a little more than two students.

In [ ]:
bars("Mean of Faculty Members per Student aggregated to country for the QS ranking", df_ranking_qs.sort_values('Country'), 'Country', 'Fac Members per Student')

We can observe that Russia obtain the best score, but as there is no standard deviation, we guess that there is only one school taken in account, and this result thus is not very relevant (same for Saudi Arabia). On the other hand, we see that Danemark has also a good ratio for faculty members, but its standard deviation is high, which implies that this ratio is very heterogeneous among the universities.

##### - In terms of Percentage of International Students

In [ ]:
qs_percInterStu_country=df_ranking_qs.loc[df_ranking_qs.groupby("Country")["% International Students"].idxmax()]
qs_percInterStu_country[['Country','University','% International Students']].set_index('Country').sort_values('% International Students',ascending=False).head()

The United Kingdom has the university with the best ratio of international students, the London School of Economics and Political Science university where 69% of their students are international.

In [ ]:
bars("Mean of International Students aggregated by country for the QS ranking", df_ranking_qs.sort_values('Country'), 'Country', '% International Students')

The best countries in term of international students are the United Kingdom, Australia, Hong Kong, Switzerland, and Austria. It is not a surprise that the United Kingdom has the best ratio of international students, as it can attract a lot of european students by its proximity and by the fact that most people speak english.

#### d) Aggregating the data by region

As for the aggregation by country, we first found by region the university with the best results in the Faculty members per Student and Percentage of International Students respectively, and we plot the mean for each region.

##### - In terms of Ratio between Faculty members and Students

In [ ]:
qs_ratioFacStu_region=df_ranking_qs.loc[df_ranking_qs.groupby("Region")["Fac Members per Student"].idxmax()]
qs_ratioFacStu_region[['Region','Country','University','Fac Members per Student']].set_index('Region').sort_values('Fac Members per Student',ascending=False)

In [ ]:
bars("Mean of Faculty Members per Student aggregated to region for the QS ranking", df_ranking_qs.sort_values('Region'), 'Region', 'Fac Members per Student')

##### - In terms of Percentage of International Students

In [ ]:
qs_percInterStu_region=df_ranking_qs.loc[df_ranking_qs.groupby("Region")["% International Students"].idxmax()]
qs_percInterStu_region[['Region','Country','University','% International Students']].set_index('Region').sort_values('% International Students',ascending=False)

In [ ]:
bars("Mean of International Students aggregated by region for the QS ranking", df_ranking_qs.sort_values('Region'), 'Region', '% International Students')

## Question 2

Obtain the 200 top-ranking universities in www.timeshighereducation.com ([ranking 2018](http://timeshighereducation.com/world-university-rankings/2018/world-ranking)). Repeat the analysis of the previous point and discuss briefly what you observed.

Using Postman, a API request was captured on the ranking, containing a json file with all the data.

In [ ]:
times_r=requests.get('https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json')
data=times_r.text

We decode the json file by keeping only the first 200 data items (each data item corresponds to an university, sorting according its ranking). Furthemore, we just extract the useful columns (name, location, rank, total number of students, percentage of international students and student/staff ratio). 

As we are asked to sort the universities according their ratio between faculty members and students, we're already converted the student/staff ratio to staff/student ratio.

In [ ]:
df_times = pd.DataFrame(json.loads(data)['data'][:200],columns=['name','location','rank','scores_overall','stats_number_students','stats_pc_intl_students','stats_student_staff_ratio'])
df_times['scores_overall'] = df_times['scores_overall'].astype(float)
df_times['stats_pc_intl_students'] = df_times['stats_pc_intl_students'].astype(str).str.replace('%','').astype(int)
df_times['stats_number_students'] = df_times['stats_number_students'].astype(str).str.replace(',','').astype(float)
df_times['stats_student_staff_ratio'] = 1/(df_times['stats_student_staff_ratio'].astype(float))
df_times.columns = ['University','Country','Rank','Score','Total Students','% International Students','Fac Members per Student']
df_times.head()


We don't have the region information in the second website so we decide to use the information present in the first one.  
In order to have a broader panel of Country-Region relation, we extract this time the overall ranking of the QS website but keeping only the Country and Region variables and we keep the unique combinations that we will join on our Times ranking dataframe.

In [ ]:
conv_to_cont = pd.DataFrame(data_QS.json()['data'],columns=['country','region']).drop_duplicates()
conv_to_cont.columns=['Country','Region'];
conv_to_cont.head()

Then, we merged the two dataframes on the country attribute.

In [ ]:
df_ranking_times=pd.merge(df_times, conv_to_cont,how='left',left_on="Country",right_on="Country")


In [ ]:
df_ranking_times[df_ranking_times.Region.isnull()]

As we can see above, two universities have no defined Region because the information about these countries was missing in the first ranking. We complete manually the missing information as just two universities are concerned and they won't appear on the barplot when we'll aggregate ranking by region otherwise.

In [ ]:
df_ranking_times.at[178,'Region']='Europe'
df_ranking_times.at[193,'Region']='Europe'

Some informations are not explicitely present in our data as the number of international students and the total number of faculty members but we can infer them with the other columns according the following formulas :

Number of international students : *0.01 x Percentage of International Students x Total number of Students*

Total number of faculty members : *Total number of students x Ratio between Faculty members and Students*

Nethertheless, we cannot infer the number of international faculty members but it is not essential for the results.

In [ ]:
df_ranking_times['International Students'] = (df_ranking_times["Total Students"] * 0.01 * df_ranking_times["% International Students"]).astype(int)
df_ranking_times['Total Faculty Members'] = (df_ranking_times["Total Students"] * df_ranking_times["Fac Members per Student"]).astype(int)
df_ranking_times.head()

#### a) Which are the best universities in term of ratio between faculty members and students ?
For the following questions, we choose to present the results the same way as for the QS ranking (see explanations in Qu.1).

In [ ]:
df_ranking_times.sort_values('Fac Members per Student', ascending=False)

In [ ]:
bars("Top-20 of the best universities in term of Faculty Members per Student for the Times ranking", df_ranking_times.sort_values('Fac Members per Student', ascending=False)[:20], 'University', 'Fac Members per Student')


#### b) Which are the best universities in term of ratio of international students?

In [ ]:
df_ranking_times.sort_values('% International Students', ascending=False)

In [ ]:
bars("Top-20 of the best universities in term of International Students per University for the Times ranking", df_ranking_times.sort_values('% International Students', ascending=False)[:20], 'University', '% International Students')

#### c) Aggregating the data by country

Again, we use the same representations as for Question 1 to answer to these questions.

##### - In terms of Ratio between Faculty members and Students

In [ ]:
times_ratioFacStu_country=df_ranking_times.loc[df_ranking_times.groupby("Country")["Fac Members per Student"].idxmax()]
times_ratioFacStu_country[['Country','University','Fac Members per Student']].set_index('Country').sort_values('Fac Members per Student',ascending=False).head()

The United Kingdom are no longer in the top-5 of the countries with the best Faculty Members per Student ratio, compared to the QS ranking.

Furthermore, we can see that the maximum ratio is much smaller than the QS ranking, which is strange since it is an official information that should not depend on any ranking.

In [ ]:
bars("Mean of Faculty Members per Student aggregated to country for the Times ranking", df_ranking_times.sort_values('Country'), 'Country', 'Fac Members per Student')

##### - In terms of Percentage of International Students

In [ ]:
times_percInterStu_country=df_ranking_times.loc[df_ranking_times.groupby("Country")["% International Students"].idxmax()]
times_percInterStu_country[['Country','University','% International Students']].set_index('Country').sort_values('% International Students',ascending=False).head()

The results are much more similar to the QS rankings compared to the previous parameter. Indeed, the United Kingdom is also the country with the best university in terms of Percentage of International Students. The Luxembourg is the second one but the University of Luxembourg is not present in the top-200 of QS ranking.

In [ ]:
bars("International Students per Country", df_ranking_times.sort_values('Country'), 'Country', '% International Students')

#### d) Aggregating the data by region

##### - In terms of Ratio between Faculty members and Students

In [ ]:
times_ratioFacStu_region=df_ranking_times.loc[df_ranking_times.groupby("Region")["Fac Members per Student"].idxmax()]
times_ratioFacStu_region[['Region','Country','University','Fac Members per Student']].set_index('Region').sort_values('Fac Members per Student',ascending=False)

The ranking of the best Regions in term of the Ration between Faculty members and Students is exactly the same as for the QS rankings, even if the values for this parameter are sensibly differents between the two rankings as we mentionned before.

Indeed, there is a huge gap between the different regions and this explained why there is no change on the top-5.  

It is also interesting to notice that there aren't universities from Latin America countries in the Times ranking.

In [ ]:
bars("Mean of Faculty Members per Student aggregated to region for the Times ranking", df_ranking_times.sort_values('Region'), 'Region', 'Fac Members per Student')


In [ ]:
times_percInterStu_region=df_ranking_times.loc[df_ranking_times.groupby("Region")["% International Students"].idxmax()]
times_percInterStu_region[['Region','Country','University','% International Students']].set_index('Region').sort_values('% International Students',ascending=False)

Except for the Latin America region which doesn't appears on the dataframe as we said before, the ranking of Region according the best universities in termns of Percentage of International Students is exactly the same between the two rankings (same country, same university and nearly the same percentage)

In [ ]:
bars("Mean of International Students aggregated by region for the Times ranking", df_ranking_times.sort_values('Region'), 'Region', '% International Students')

We save our current results in a pickle file.

In [ ]:
pickle.dump( df_ranking_qs , open( "qs.p", "wb" ) )
pickle.dump( df_ranking_times , open( "times.p", "wb") )

In [ ]:
df_ranking_qs = pickle.load(open( "qs.p", "rb" ))
df_ranking_times = pickle.load(open( "times.p", "rb" ))

## Question 3


Merge the two DataFrames created in questions 1 and 2 using university names. Match universities' names as well as you can, and explain your strategy. Keep track of the original position in both rankings.

First, we try a naive inner (by default) merge on the "University" columns and we obtain 105 rows, meaning only about half of the university names matched.

In [ ]:
df_ranking_qs.merge(df_ranking_times,on="University")

So we take a closer look at the list of names for each ranking :

In [ ]:
pd.DataFrame({"qs":df_ranking_qs["University"],"times":df_ranking_times["University"]})

We notice a lot of universities have their initials in parenthesis after their name in the QS ranking, so we start by removing those parenthesis.

In [ ]:
df_ranking_qs["University"] = df_ranking_qs["University"].str.replace(r"\(.*\)","").str.strip()

In [ ]:
df_ranking_qs.merge(df_ranking_times,on="University").shape[0]

We only obtain a slight improvement of 10 more matches. 
This is definitely not enough and even by further study of the 2 sets of names, there doesn't seem to be any other way to bump this number up.

So we try a new approach : the standard python library *difflib* offers an effective way to measure string similarity with its *SequenceMatcher* class.
We compare the two lists of names two by two, and if the best match is above a certain threshold of similarity, we map the two names together.
The similarity threshold is calibrated after some trials in order to make sure we don't wrongly match too many names (high enough) but still have enough flexibility (low enough) to match all the universities that appear on both rankings.

In [ ]:
from difflib import SequenceMatcher

df_ranking_qs = pickle.load(open( "qs.p", "rb" )) #just to reset our previous modification of the names

name_map = {}
threshold = 0.85

for name1 in df_ranking_qs["University"]:
    best_ratio = 0
    match = ""
    for name2 in df_ranking_times["University"]:
        ratio = SequenceMatcher(None,name1,name2).ratio()
        if ratio > best_ratio:       # we find the maximum similarity
            best_ratio = ratio
            match = name2
    if (best_ratio > threshold):     # if it's superior to our threshold we add that couple to the mapping
        name_map[name1] = match

print("Nb of matches :",len(name_map))
name_map

The result is much more pleasing : we obtain 149 matches. But going through that mapping, we notice some names appear several times on the value side, which means we wrongly mapped some universities. Let's see which ones :

In [ ]:
l = list(name_map.values())
duplicates = [x for x in l if l.count(x) > 1]
duplicates

Only 5 universities appear twice, which seems reasonable to correct "by hand".

In [ ]:
inv_names = {}
for k, v in name_map.items():
    inv_names[v] = inv_names.get(v, [])
    inv_names[v].append(k)

inv_duplicates = {d:inv_names[d] for d in duplicates}
inv_duplicates

With this reversed mapping of names, we can see that, for example 'Eindhoven University of Technology' has been mapped to both 'Eindhoven University of Technology' and 'Vienna University of Technology', but the Vienna University of Technology does not appear in the Times ranking. A quick check shows that it is the same for the 4 other cases, so we simply correct these 5 entries in our mapping by deleting them.

In [ ]:
for k,v in inv_duplicates.items():
    for uni in v:
        if uni != k:
            del(name_map[uni])

print("Nb of matches :",len(name_map))
name_map

Finally we can do an inner merge between the two rankings on the university name column, and we only show here the basic information, mostly respective ranks, for more clarity.

In [ ]:
df_ranking_qs["University"] = [name_map[uni] if uni in name_map.keys() else uni for uni in df_ranking_qs["University"]]

df_merged = df_ranking_qs.merge(df_ranking_times.drop(["Country","Region"],axis=1),on="University",suffixes=("_qs","_times"))
df_merged[["University","Rank_qs","Rank_times","Country","Region"]]

## Question 4
Find useful insights in the data by performing an exploratory analysis. Can you find a strong correlation between any pair of variables in the dataset you just created? Example: when a university is strong in its international dimension, can you observe a consistency both for students and faculty members?

First of all, we decided to plot the correlation matrix but not for all the columns of the merged dataframe. We kept only the most important attributes which seemed interesting to compare : the overall score, the total number of students, the ratio between the Faculty members and Students and the percentage of international students.

In order to facilitate the interpretation of the resulting correlation matrix, we plot this symmetrical matrix according a color map (red for positive correlation and blue for negative correlation).

In [ ]:
corr_matrix=df_merged[['Score_qs','Total Students_qs','Fac Members per Student_qs','% International Students_qs','Score_times','Total Students_times','Fac Members per Student_times','% International Students_times']].corr()

In [ ]:
def corr_plot(corr):   
    plt.figure(figsize=(10, 10))
    plt.imshow(corr, cmap='RdBu_r', interpolation='none', aspect='auto', vmin=-1, vmax=1)
    plt.colorbar()
    plt.xticks(range(len(corr)), corr.columns, rotation='vertical')
    plt.yticks(range(len(corr)), corr.columns);
    plt.suptitle('Stock Correlations Heat Map', fontsize=15, fontweight='bold')
    plt.show()

In [ ]:
corr_plot(corr_matrix)

As we expected, there is a strong positive correlation between two rankings for the same parameters, as we can see with the upper (or lower because the matrix is symmetrical) red diagonal.

It is also really interesting to see that there is absolutely no correlation between the number of students in the university and its score, for the both rankings.

We also decided to plot the score attributed by the QS ranking versus the score attributed by the Times ranking to see if there was any interesting underlying structure.

In [ ]:
from bokeh.plotting import figure, show, output_notebook, ColumnDataSource
from bokeh.models import HoverTool
from bokeh.palettes import Set2
from bokeh.transform import factor_cmap
output_notebook()

In [ ]:
source=ColumnDataSource(df_merged)

hover = HoverTool(tooltips=[
    ("University", "@University"),
    ("QS Rank", "@Score_qs"),
    ("Times Rank", "@Score_times"),
    ("Region", "@Region"),
    
])
p = figure(title="HP vs MPG (shaded by CYL)", tools=[hover])
p.circle(x='Score_qs', y='Score_times',source=df_merged,size=12,line_color="white",alpha=0.7, fill_color=factor_cmap('Region', palette=Set2.get(5), factors=df_merged['Region'].unique()))
p.line([49,98], [54,93], alpha=0.6, color='red')
p.xaxis.axis_label = 'Score QS'
p.yaxis.axis_label = 'Score Times'
show(p)

First of all, there is a strong positive correlation between the two rankings which is logical. But we can notice some really interesting observations.

Indeed we can see that the Asian universities have mostly a better score than in the QS ranking than in the Times ranking.

## Question 5

Can you find the best university taking in consideration both rankings? Explain your approach.

Considering our data, the best way to combine both rankings seems to be the simplest : averaging the 2 different scores. Before that, we just need to standardize those scores by substracting the mean and dividing by the standard deviation. We then add 5 to get a score out of 10 with 5 being the median, so that all scores are positive and we can compute the average.

In [ ]:
std_score_qs = (df_merged['Score_qs'] - df_merged['Score_qs'].mean())/df_merged['Score_qs'].std() + 5
std_score_times = (df_merged['Score_times'] - df_merged['Score_times'].mean())/df_merged['Score_times'].std() + 5

df_merged["Average score"] = (std_score_qs+std_score_times)/2

df_merged.sort_values("Average score",ascending=False)[["University","Average score"]]

Using this score as a ranking, the MIT is the best university.